# Gestalt Principles and CNNs
- Submitted by Sankalp Mittal (220963)

# Part a
In this part we are required to repriduce the study design using inception v4 networks instead of inception v3 networks

## InceptionV4 Implementation
I am using the following [github](https://github.com/kentsommer/keras-inceptionV4/tree/master) repository for the code for InceptionV4 model implementation as it was not available in the keras library

In [1]:
# from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
# from tensorflow.keras.models import Model
import inception_v4

2025-04-02 17:47:59.968964: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-02 17:48:00.005907: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-02 17:48:00.229182: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-02 17:48:00.327003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743596280.496741    3179 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743596280.53

In [2]:
def build_inceptionV4(num_classes=1001, dropout_prob=0.2, weights='imagenet', include_top=True):
    try:
        return inception_v4.create_model(num_classes, dropout_prob, weights, include_top)
    except ValueError:
        try:
            return inception_v4.create_model(num_classes, dropout_prob, weights, include_top)
        except ValueError:
            return inception_v4.create_model(num_classes, dropout_prob, weights, include_top)

Just a check to see model is being built without any problems

In [ ]:
model = build_inceptionV4(num_classes=1001, dropout_prob=0.2, weights='imagenet')

# Generate Data
We generate the datasets using the `createImage` file using the following commands in the file directly
```
generate_global_symmetric_data()
# generate_local_symmetric_data()
# generate_count_object_data()
# generate_count_type_data()
```
We only generate global symmetry data as only that is asked in the question, even that comes out to be approximately __8GB__ in size just for _ds1_ and _ds2_ after which my system started killing the process, so I generated the datasets sequentially so that the run goes smoothly.

As even that was not working I just generated a dataset of size 10% of what was mentioned in the paper. 

# Part b
In this part we try to reproduce the results produced for InceptionV3 models with InceptionV4 for the case of detecting symmetry in images

We will use the same helper functions for training as in the case of InceptionV3

In [3]:
import train_incep3
import os
import numpy as np

## Training Run 1
This will run on ds1

In [4]:
ds = "data\symmetry_global\ds1"
resume_model = None
im_noise = False

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3179/1192969502.py:1: SyntaxWarning: invalid escape sequence '\s'
  ds = "data\symmetry_global\ds1"


In [5]:
nb_epoch2 = 10

if not os.path.exists('models'):
    os.mkdir('models')

# load data
tx, ty = train_incep3.read_dataset(ds+'/train.data')
vx, vy = train_incep3.read_dataset(ds+'/valid.data')

nTrainSample = len(ty)

tx = train_incep3.preprocessing_img(tx)
vx = train_incep3.preprocessing_img(vx)
print("tx.shape: ", tx.shape)
# print("ty.shape: ", ty.shape)
print("vx.shape: ", vx.shape)
# print("vy.shape: ", vy.shape)

tx = np.asarray(tx)[:, np.newaxis, :, :]
vx = np.asarray(vx)[:, np.newaxis, :, :]
tx = np.repeat(tx, 3, axis=1)
vx = np.repeat(vx, 3, axis=1)

    
ty = train_incep3.to_categorical(ty, 2)
vy = train_incep3.to_categorical(vy, 2)

## build model
print("About to build model")
model = build_inceptionV4(num_classes=2)

datagen = train_incep3.ImageDataGenerator(
    preprocessing_function=train_incep3.add_random_noise if im_noise else None,
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.02,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.02,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images
    

if resume_model is not None:
    print('Resume model: ', resume_model)
    model.load_weights(resume_model)



# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


filepath2="models/"+ds+"_incep3-{epoch:02d}-{val_acc:.3f}.weights.h5"
checkpoint2 = train_incep3.callbacks.ModelCheckpoint(filepath2, monitor='val_acc', verbose=0, save_best_only=True, mode='max', save_weights_only=True)

early_stopper   = train_incep3.EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=40)

callbacks_list2 = [early_stopper, checkpoint2]

# we train our model 
model.fit(datagen.flow(tx, ty, batch_size=40, shuffle=True),
                    steps_per_epoch=nTrainSample//40,
                    epochs=nb_epoch2,
                    validation_data=(vx, vy),
                    callbacks=callbacks_list2)

        
# serialize last weights to HDF5
model.save_weights(ds+"_incep3-%d.weights.h5"%(nb_epoch2))

print("Saved last weights to disk")

tx.shape:  (800, 299, 299)
vx.shape:  (800, 299, 299)
About to build model


E0000 00:00:1743596305.533875    3179 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1743596305.537005    3179 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/sankalp/CGS786_assn3/final_implementation/inception_v4.py:279: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '
/home/san

Epoch 1/10


: 